In [72]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import requests
import matplotlib.pyplot as plt
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from urllib.parse import unquote
from urllib.parse import urlencode
import time
import re
import math
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import string
from sklearn.preprocessing import normalize
from scipy.spatial import distance
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
from sklearn.model_selection import train_test_split
# Exercise 5.1.3 Clustering using NLTK KMean
# cosine distance is calculated
import json
from numpy.random import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from nltk.corpus import stopwords
from nltk.cluster import KMeansClusterer, \
cosine_distance 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
from collections import defaultdict
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_validate
from sklearn import svm
from gensim.models.doc2vec import TaggedDocument
from random import shuffle
from gensim.models import doc2vec

## Code for scraping description, difficulty level, rate and language

In [12]:
def get_info(link):
    result = []
    page = requests.get("https://www.coursera.org"+link)
    soup = BeautifulSoup(page.content, 'html.parser')

    try:
        des=soup.select("div.content-inner")
        result.append(des[0].get_text())
    except Exception as e:
        result.append("no information")
        
    level = soup.select("h4.H4_1k76nzj-o_O-weightBold_uvlhiv-o_O-bold_1byw3y2.m-b-0.m-t-1s")
    try:
        result.append(level[0].get_text())
    except Exception as e:
        result.append("no level")
    
    rate = soup.select("span.H4_1k76nzj-o_O-weightBold_uvlhiv-o_O-bold_1byw3y2.m-l-1s.m-r-1.m-b-0")
    try:
        result.append(rate[0].get_text())
    except Exception as e:
        result.append("no rate")
    
    language = soup.select("h4.H4_1k76nzj-o_O-weightBold_uvlhiv-o_O-bold_1byw3y2.m-b-0")
    try:
        result.append(language[-1].get_text())
    except Exception as e:
        result. append("no language")
        
    return result

## Example of get_info function

In [ ]:
get_info("/learn/what-is-a-proof")

## Code for getting title and course page links

In [78]:
def get_T_L(topic):
    whole_links=[]
    links=[]
    title = []
    title1=[]
    difficulty = []
    difficulty1 = []
    i = 0
    page = requests.get("https://www.coursera.org/courses?query="+topic+"%20science&indices%5Bprod_all_products_custom_ranking_revenuelast28d%5D%5Bpage%5D=6")
    soup = BeautifulSoup(page.content, 'html.parser')
    result=soup.select("h3.rc-NumberOfResultsSection.body-2-text span")
    result_text = result[0].get_text()
    reg = "\d+"
    number = re.findall(reg,result_text)
    number = int(number[0])
    if number > 1000:
        i = 100
    elif number%10 == 0:
        i = number/10
    else:
        i = math.floor(number/10 +1)
#     print(i)
    
    x = 1
    while x <= i:
        page_links = []
#         print(x)    
        page = requests.get("https://www.coursera.org/courses?query="+topic+"%20science&indices%5Bprod_all_products_custom_ranking_revenuelast28d%5D%5Bpage%5D=" + str(x))
        soup = BeautifulSoup(page.content, 'html.parser')
        div_links=soup.select("div.tab-contents div.ais-InfiniteHits ul.ais-InfiniteHits-list li.ais-InfiniteHits-item a")
        for idx, link in enumerate(div_links):
            if link.get('href').find("specializations")>-1 or link.get('href').find("degrees")>-1:
                page_links.append("n")
            else:
                get_link=link.get('href')
                page_links.append(link.get('href'))
           
    
        if "/learn/causal-effects" in page_links and x >1:
            continue
        else:
            x+=1
            whole_links += page_links
            divs=soup.select("#rendered-content .ais-InfiniteHits-list > .ais-InfiniteHits-item h2")
            for idx, div in enumerate(divs):
                title.append(div.get_text().lower())

    for x in range(0,len(whole_links)):
        if whole_links[x]!="n":
            title1.append(title[x])
    for link in whole_links:
        if link != "n":
            links.append(link)
    return title1, links

In [79]:
title, link = get_T_L("social")

In [ ]:
title

In [ ]:
link

## Code to scrap all information of descriptions, levels, rates and languages

In [83]:
des = []
i = 0
for x in link:
    i += 1
#     print(i)
    des.append(get_des(x))

In [ ]:
des

## Code to write these information into Excel file

In [ ]:
import xlwt
file = xlwt.Workbook()
table = file.add_sheet('sheet1')
table.write(0,0,"course")
table.write(0,1,"description")
table.write(0,2,"level")
table.write(0,3,"rate")
table.write(0,4,"language")

for i in range(1,len(data)+1):
    table.write(i,0,title[i-1][0])
    table.write(i,1,des[i-1][0][0])
    table.write(i,2,des[i-1][0][1])
    table.write(i,3,des[i-1][0][2])
    table.write(i,4,des[i-1][0][3])

file.save('C:\WORK AND STUDY\Semester 2\BIA-660\Final\Social Science.xls')
print("finish")

## Import train data

In [ ]:
data=pd.read_csv("C:/WORK AND STUDY/Semester 2/BIA-660/Final/concat_train.csv",header=0)
data

## generate tfidf matrix

In [18]:
tfidf_vect = TfidfVectorizer() 

# with stop words removed
#tfidf_vect = TfidfVectorizer(stop_words="english") 

# generate tfidf matrix
dtm= tfidf_vect.fit_transform(data["description"])

print("type of dtm:", type(dtm))
print("size of tfidf matrix:", dtm.shape)

type of dtm: <class 'scipy.sparse.csr.csr_matrix'>
size of tfidf matrix: (848, 10519)


## Train and test LDA model and check the performance

In [31]:
X_train, X_test = train_test_split(\
                data, test_size=0.3, random_state=0)
tf_vectorizer = CountVectorizer(max_df=0.90, \
                min_df=2, stop_words='english')
tf = tf_vectorizer.fit_transform(X_train["description"])
tf_test = tf_vectorizer.transform(X_test["description"])


single_label = []
for x in X_test["level"]:
    if not x in single_label:
        single_label.append(x)

num_topics = 3
lda = LatentDirichletAllocation(n_components=num_topics, \
                                max_iter=50,verbose=1,
                                 n_jobs=8,random_state=0).fit(tf)

topic_assign=lda.transform(tf_test)
sort_probs = np.argsort(topic_assign)
predicted = []
for prob in sort_probs:
    predicted.append(prob[2])
confusion_df = pd.DataFrame(list(zip(X_test["level"].values, predicted)),\
                        columns = ["label", "cluster"])
vote_result = pd.crosstab( index=confusion_df.cluster, columns=confusion_df.label)
print(vote_result)
cluster_dict = {}

for x in range(3):
    Max = 0
    for label in single_label:
        if vote_result[label][x] > Max:
            Max = vote_result[label][x]
            cluster_name = label
    cluster_dict[x] = cluster_name

print("Cluster 0: " + cluster_dict[0])
print("Cluster 1: " + cluster_dict[1])
print("Cluster 2: " + cluster_dict[2])
predicted_target=[cluster_dict[i] \
                  for i in predicted]
print(metrics.classification_report\
      (X_test["level"], predicted_target))

iteration: 1 of max_iter: 50
iteration: 2 of max_iter: 50
iteration: 3 of max_iter: 50
iteration: 4 of max_iter: 50
iteration: 5 of max_iter: 50
iteration: 6 of max_iter: 50
iteration: 7 of max_iter: 50
iteration: 8 of max_iter: 50
iteration: 9 of max_iter: 50
iteration: 10 of max_iter: 50
iteration: 11 of max_iter: 50
iteration: 12 of max_iter: 50
iteration: 13 of max_iter: 50
iteration: 14 of max_iter: 50
iteration: 15 of max_iter: 50
iteration: 16 of max_iter: 50
iteration: 17 of max_iter: 50
iteration: 18 of max_iter: 50
iteration: 19 of max_iter: 50
iteration: 20 of max_iter: 50
iteration: 21 of max_iter: 50
iteration: 22 of max_iter: 50
iteration: 23 of max_iter: 50
iteration: 24 of max_iter: 50
iteration: 25 of max_iter: 50
iteration: 26 of max_iter: 50
iteration: 27 of max_iter: 50
iteration: 28 of max_iter: 50
iteration: 29 of max_iter: 50
iteration: 30 of max_iter: 50
iteration: 31 of max_iter: 50
iteration: 32 of max_iter: 50
iteration: 33 of max_iter: 50
iteration: 34 of ma

## Train and test for the MultinomialNB Model

In [21]:


X_train, X_test, y_train, y_test = train_test_split(\
                dtm, data["level"], test_size=0.3, random_state=0)

# train a multinomial naive Bayes model using the training data
clf = MultinomialNB().fit(X_train, y_train)

# predict the news group for the test dataset
predicted=clf.predict(X_test)

## Check the performance of MultinomialNB

In [22]:
labels=sorted(data["level"].unique())

# calculate performance metrics. 
# Support is the number of occurrences of each label

precision, recall, fscore, support=\
     precision_recall_fscore_support(\
     y_test, predicted, labels=labels)

print("labels: ", labels)
print("precision: ", precision)
print("recall: ", recall)
print("f-score: ", fscore)
print("support: ", support)

# another way to get all performance metrics
print(classification_report\
      (y_test, predicted, target_names=labels))

labels:  ['Advanced Level', 'Beginner Level', 'Intermediate Level']
precision:  [0.         0.66363636 0.71428571]
recall:  [0.         0.96688742 0.27777778]
f-score:  [0.         0.78706199 0.4       ]
support:  [ 14 151  90]
                    precision    recall  f1-score   support

    Advanced Level       0.00      0.00      0.00        14
    Beginner Level       0.66      0.97      0.79       151
Intermediate Level       0.71      0.28      0.40        90

         micro avg       0.67      0.67      0.67       255
         macro avg       0.46      0.41      0.40       255
      weighted avg       0.65      0.67      0.61       255



C:\Users\Laysly\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Train and test for the SVM Model

In [24]:
X_train, X_test, y_train, y_test = train_test_split(\
                dtm, data["level"], test_size=0.25, random_state=0)

# train a multinomial naive Bayes model using the training data
clf = svm.LinearSVC().fit(X_train, y_train)

# predict the news group for the test dataset
predicted=clf.predict(X_test)



## Check the performance of SVM

In [25]:
labels=sorted(data["level"].unique())

# calculate performance metrics. 
# Support is the number of occurrences of each label

precision, recall, fscore, support=\
     precision_recall_fscore_support(\
     y_test, predicted, labels=labels)

print("labels: ", labels)
print("precision: ", precision)
print("recall: ", recall)
print("f-score: ", fscore)
print("support: ", support)

# another way to get all performance metrics
print(classification_report\
      (y_test, predicted, target_names=labels))

labels:  ['Advanced Level', 'Beginner Level', 'Intermediate Level']
precision:  [1.         0.76595745 0.64179104]
recall:  [0.33333333 0.85714286 0.58108108]
f-score:  [0.5        0.80898876 0.60992908]
support:  [ 12 126  74]
                    precision    recall  f1-score   support

    Advanced Level       1.00      0.33      0.50        12
    Beginner Level       0.77      0.86      0.81       126
Intermediate Level       0.64      0.58      0.61        74

         micro avg       0.73      0.73      0.73       212
         macro avg       0.80      0.59      0.64       212
      weighted avg       0.74      0.73      0.72       212



## Predict the Mixed level

In [27]:
data2=pd.read_csv("C:/WORK AND STUDY/Semester 2/BIA-660/Final/concat.csv",header=0)
data2 = data2[data2.level == "Mixed Level"]
docs_new = data2["description"]

X_new_tfidf = tfidf_vect.transform(docs_new)
predict = clf.predict(X_new_tfidf)
predict

array(['Beginner Level', 'Intermediate Level', 'Beginner Level',
       'Beginner Level', 'Beginner Level', 'Beginner Level',
       'Beginner Level', 'Beginner Level', 'Beginner Level',
       'Beginner Level', 'Beginner Level', 'Intermediate Level',
       'Beginner Level', 'Beginner Level', 'Intermediate Level',
       'Beginner Level', 'Beginner Level', 'Beginner Level',
       'Beginner Level', 'Beginner Level', 'Beginner Level',
       'Beginner Level', 'Beginner Level', 'Beginner Level',
       'Beginner Level', 'Beginner Level', 'Beginner Level',
       'Intermediate Level', 'Beginner Level', 'Beginner Level',
       'Beginner Level', 'Beginner Level', 'Beginner Level',
       'Intermediate Level', 'Beginner Level', 'Beginner Level',
       'Intermediate Level', 'Beginner Level', 'Beginner Level',
       'Beginner Level', 'Beginner Level', 'Intermediate Level',
       'Beginner Level', 'Beginner Level', 'Beginner Level',
       'Beginner Level', 'Intermediate Level', 'Beginner 

## Add the predict value to the mixed level

In [28]:
data3=pd.read_csv("C:/WORK AND STUDY/Semester 2/BIA-660/Final/concat.csv",header=0)
i = 0
for x in range(len(data3)):
    if data3["level"][x] == "Mixed Level":
        data3["level"][x] = "Mixed Level " + "(" + predict[i] + ")"
        i += 1

C:\Users\Laysly\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [29]:
data3

,course,description,level,rate,language
0,sustainable business enterprises,This course will explore current challenges an...,Mixed Level (Beginner Level),4.5,English
1,excel skills for business: essentials,In this first course of the specialization Exc...,Mixed Level (Intermediate Level),4.9,English
2,business strategy,In this course you will learn how organization...,Mixed Level (Beginner Level),4.8,English
3,Exploring and Producing Data for Business Deci...,This course provides an analytical framework t...,Beginner Level,4.8,English
4,"Brand Management: Aligning Business, Brand and...",Professor Nader Tavassoli of London Business S...,Beginner Level,4.9,English
5,excel skills for business: intermediate i,Spreadsheet software remains one of the most u...,Intermediate Level,4.9,English
6,digital business models,Digital business models are disrupting 50-year...,Mixed Level (Beginner Level),4.5,English
7,english for business and entrepreneurship,Welcome to English for Business and Entreprene...,Beginner Level,4.8,English
8,foundations of business strategy,"Develop your ability to think strategically, a...",Mixed Level (Beginner Level),4.8,English
9,global impact: business ethics,Global business ethics is the study and analys...,Intermediate Level,4.8,English


## Code to write these information into Excel file

In [ ]:
file = xlwt.Workbook()
table = file.add_sheet('sheet1')
table.write(0,0,"course")
table.write(0,1,"description")
table.write(0,2,"level")
table.write(0,3,"rate")
table.write(0,4,"language")

for i in range(0,len(data3)):
    table.write(i + 1,0,data3["course"][i])
    table.write(i + 1,1,data3["description"][i])
    table.write(i + 1,2,data3["level"][i])
    table.write(i + 1,3,data3["rate"][i])
    table.write(i + 1,4,data3["language"][i])

file.save('C:\WORK AND STUDY\Semester 2\BIA-660\Final\Combine.xls')
print("finish")

## Read the final data

In [32]:
data=pd.read_csv("C:\WORK AND STUDY\Semester 2\BIA-660\Final\Combine.csv",
header=0)

In [33]:
wl = WordNetLemmatizer()
stop_words = stopwords.words('english')

## Code to tokenize

In [39]:
def tokenize(doc, lemmatized=False, no_stopword=False):
    tokens =[]
    tokens = nltk.word_tokenize(doc.lower()) 
    tokens=[token.strip(string.punctuation) for token in tokens]
    # remove empty tokens
    tokens=[token.strip() for token in tokens \
            if token.strip()!='']
    if no_stopword==True:
        tokens=[token for token in tokens \
                if not token in stop_words]
    if lemmatized==True:
        tokens=[wl.lemmatize(token) for token in tokens]
    return tokens

## Code to get cosine similarity matrix

In [40]:
def get_similarity(data, lemmatized=False, no_stopword=False):

    data_tokens={idx:nltk.FreqDist(tokenize(des,lemmatized, no_stopword)) \
             for idx,des in enumerate(data)}
    dtm=pd.DataFrame.from_dict(data_tokens, \
                           orient="index" )
    dtm=dtm.fillna(0)

    tf=dtm.values


    data_len=tf.sum(axis=1)

    np.shape(tf)

    tf=np.divide(tf, data_len[:,None])
    tf

    df=np.where(tf>0,1,0)

    idf=np.log(np.divide(len(data), \
            np.sum(df, axis=0)))+1
    
    smoothed_idf=np.log(np.divide(len(data)+1, np.sum(df, axis=0)+1))+1
    

    
    tf_idf=normalize(tf*idf)
    
    
    smoothed_tf_idf=normalize(tf*smoothed_idf+0.008)
    print(smoothed_tf_idf)

    similarity=1-distance.squareform(distance.pdist(smoothed_tf_idf, 'cosine'))

        
    return similarity

## Transform the final data to list, and form the "doc" list for input

In [41]:
course = data["course"].tolist()
des = data["description"].tolist()
level = data["level"].tolist()
rate = data["rate"].tolist()
doc = []
for i in range(len(des)):
    doc.append(course[i] + des[i])

## Get cosine similarity matrix and sort it in decending order according to index

In [69]:
sim=get_similarity(doc, lemmatized=True, no_stopword=True)
sim
sim_sort=np.argsort(sim)[:,::-1]
sim_sort

[[0.18339414 0.12789875 0.05810101 ... 0.00705986 0.00705986 0.00705986]
 [0.0072268  0.03962342 0.0072268  ... 0.0072268  0.0072268  0.0072268 ]
 [0.04398372 0.08267792 0.00733624 ... 0.00733624 0.00733624 0.00733624]
 ...
 [0.50598622 0.00478856 0.00478856 ... 0.00478856 0.00478856 0.00478856]
 [0.00650278 0.04476341 0.00650278 ... 0.00650278 0.00650278 0.00650278]
 [0.0073135  0.02617626 0.0073135  ... 0.03918328 0.03918328 0.03918328]]


array([[1.        , 0.80010228, 0.85204712, ..., 0.61787806, 0.72736831,
        0.81879947],
       [0.80010228, 1.        , 0.82756312, ..., 0.54324877, 0.73873622,
        0.82682431],
       [0.85204712, 0.82756312, 1.        , ..., 0.56445304, 0.75550193,
        0.84628204],
       ...,
       [0.61787806, 0.54324877, 0.56445304, ..., 1.        , 0.48867637,
        0.55369259],
       [0.72736831, 0.73873622, 0.75550193, ..., 0.48867637, 1.        ,
        0.75744944],
       [0.81879947, 0.82682431, 0.84628204, ..., 0.55369259, 0.75744944,
        1.        ]])

array([[   0,  123,   41, ..., 1317,  911, 1382],
       [   1,    5,   12, ..., 1278,  911, 1382],
       [   2,  108,  949, ..., 1278,  911, 1382],
       ...,
       [1518,    0, 1240, ..., 1278, 1317, 1382],
       [1519, 1281, 1246, ..., 1278,  911, 1382],
       [1520, 1246, 1281, ..., 1317,  911, 1382]], dtype=int64)

## Code for recommendation system with cosine similarity

In [58]:
def rec_sim(course_name, current_level):
    equal = ""
    higher = ""
    if current_level == "Beginner":
        equal = "Beginner Level"
        higher = "Intermediate Level"
        
    elif current_level == "Intermediate":
        equal = "Intermediate Level"
        higher = "Advanced Level"
        
    elif current_level == "'Advanced":
        equal = "Advanced Level"
        higher = "Advanced Level"
    
    index = course.index(course_name)
    sim_list = sim_sort[index]
    equal_count = 0
    higher_count = 0
    mix_count = 0
    sim_index = 0
    result = []
    while equal_count < 5 or higher_count < 5 or mix_count < 5:
        sim_index += 1
        course_index = sim_list[sim_index]
        if level[course_index] == equal:
            if equal_count < 5:
                equal_count += 1
                result.append((course[course_index],level[course_index],rate[course_index]))
            
        elif level[course_index] == higher:
            if higher_count < 5:
                higher_count += 1
                result.append((course[course_index],level[course_index],rate[course_index]))

        elif "Mixed Level" in level[course_index]:
            if mix_count < 5:
                mix_count += 1
                result.append((course[course_index],level[course_index],rate[course_index]))

    return result

## Example for "Python Data Structures"

In [59]:
result = rec_sim("python data structures", "Beginner")
result

[('python data representations', 'Beginner Level', 4.7),
 ('python programming essentials', 'Beginner Level', 4.8),
 ('using python to access web data', 'Mixed Level (Beginner Level)', 4.8),
 ('python for data science', 'Beginner Level', 4.6),
 ('python functions, files, and dictionaries', 'Beginner Level', 4.8),
 ('programming for everybody (getting started with python)',
  'Mixed Level (Beginner Level)',
  4.8),
 ('python data analysis', 'Beginner Level', 4.7),
 ('data collection and processing with python', 'Intermediate Level', 4.7),
 ('introduction to data science in python', 'Intermediate Level', 4.5),
 ('python classes and inheritance', 'Intermediate Level', 4.7),
 ('using databases with python', 'Mixed Level (Intermediate Level)', 4.8),
 ('data structures', 'Mixed Level (Intermediate Level)', 4.7),
 ('data visualization with python', 'Intermediate Level', 4.6),
 ('applied plotting, charting & data representation in python',
  'Intermediate Level',
  4.5),
 ('python and statisti

## Code for multi-input recommendation with cosine similarity

In [60]:
def mul_rec(course, current_level):
    rec_result = []
    result = []
    for x in course:
        rec_result.append(rec_sim(x, current_level))
    i = len(rec_result)
    count = 0
    for y in range(0, i):
        result += rec_result[y] 
    final = []
    for x in result:
        if not x in final:
            final.append(x)
    return final

## Example for "Machine Learning" and "Feature Engineering"

In [61]:
a = mul_rec(["machine learning", "feature engineering"], "Intermediate")
a

[('machine learning foundations: a case study approach',
  'Mixed Level (Intermediate Level)',
  4.6),
 ('machine learning with python', 'Mixed Level (Beginner Level)', 4.7),
 ('how to win a data science competition: learn from top kagglers',
  'Advanced Level',
  4.7),
 ('launching into machine learning', 'Intermediate Level', 4.6),
 ('how google does machine learning', 'Intermediate Level', 4.6),
 ('machine learning for data analysis',
  'Mixed Level (Intermediate Level)',
  4.2),
 ('internet giants: the law and economics of media platforms',
  'Mixed Level (Beginner Level)',
  4.8),
 ('be persuasive: write a convincing position paper or policy advice (project-centered course)',
  'Mixed Level (Beginner Level)',
  4.5),
 ('big data applications: machine learning at scale', 'Advanced Level', 3.8),
 ('advanced machine learning and signal processing', 'Advanced Level', 4.6),
 ('fundamentals of machine learning in finance', 'Intermediate Level', 3.6),
 ('guided tour of machine learning i

## Code to generate word vector and document vector by doc2vec

In [63]:
des_tokens=[tokenize(description,True, True) \
             for description in des]

docs=[TaggedDocument(des_tokens[i], [str(i)]) for i in range(len(des_tokens)) ]


model = doc2vec.Doc2Vec(dm=1, min_count=5, window=5, size=200, workers=4)


model.build_vocab(docs) 

for epoch in range(30):

    shuffle(docs)

    model.train(docs, total_examples=len(docs), epochs=1)

C:\Users\Laysly\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:580: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


## Code for getting similarity score for a specific course tag

In [64]:
def get_docvec_sim(tag):
    sim = []
    for x in range(len(des)):
        sim.append(model.docvecs.similarity(str(tag), str(x)))
    sim_sort = sorted(range(len(sim)), key=lambda k: sim[k])
    sim_sort.reverse()
    return sim_sort

## Code for recommendation system based on document vector model

In [65]:
def rec_docvec(course_name, current_level):
    equal = ""
    higher = ""
    if current_level == "Beginner":
        equal = "Beginner Level"
        higher = "Intermediate Level"
        result = rec_docvec("python basics", "Beginner")
result
    elif current_level == "Intermediate":
        equal = "Intermediate Level"
        higher = "Advanced Level"
        
    elif current_level == "'Advanced":
        equal = "Advanced Level"
        higher = "Advanced Level"
    print(equal, higher)
    
    index = course.index(course_name)
    sim_list = get_docvec_sim(index)
    equal_count = 0
    higher_count = 0
    mix_count = 0
    sim_index = 0
    result = []
    while equal_count < 5 or higher_count < 5 or mix_count < 5:
        sim_index += 1
        course_index = sim_list[sim_index]
        if level[course_index] == equal:
            if equal_count < 5:
                equal_count += 1
                result.append((course[course_index],level[course_index],rate[course_index]))
            
        elif level[course_index] == higher:
            if higher_count < 5:
                higher_count += 1
                result.append((course[course_index],level[course_index],rate[course_index]))

        elif "Mixed Level" in level[course_index]:
            if mix_count < 5:
                mix_count += 1
                result.append((course[course_index],level[course_index],rate[course_index]))

    return result

## Example for "Python Basics"

In [71]:
result = rec_docvec("python basics", "Beginner")
result

Beginner Level Intermediate Level


[('python functions, files, and dictionaries', 'Beginner Level', 4.8),
 ('python data structures', 'Mixed Level (Beginner Level)', 4.9),
 ('python data analysis', 'Beginner Level', 4.7),
 ('using python to access web data', 'Mixed Level (Beginner Level)', 4.8),
 ('python classes and inheritance', 'Intermediate Level', 4.7),
 ('python data representations', 'Beginner Level', 4.7),
 ('introduction to data science in python', 'Intermediate Level', 4.5),
 ('python for data science', 'Beginner Level', 4.6),
 ('using databases with python', 'Mixed Level (Intermediate Level)', 4.8),
 ('data processing using python', 'Beginner Level', 4.4),
 ('programming for everybody (getting started with python)',
  'Mixed Level (Beginner Level)',
  4.8),
 ('data collection and processing with python', 'Intermediate Level', 4.7),
 ('app design and development for ios',
  'Mixed Level (Intermediate Level)',
  4.1),
 ('applied text mining in python', 'Intermediate Level', 4.2),
 ('algorithms, part ii', 'Inter

## Code for multi-input recommendation with document vector

In [66]:
def mul_rec_docvec(course, current_level):
    rec_result = []
    result = []
    for x in course:
        rec_result.append(rec_docvec(x, current_level))
    i = len(rec_result)
    
    count = 0
    result = []
    for y in range(0, i):
        result += rec_result[y] 
    final = []
    for x in result:
        if not x in final:
            final.append(x)
    return final

## Example for "Machine Learning" and "Feature Engineering"

In [67]:
b = mul_rec_docvec(["machine learning", "feature engineering"], "Intermediate")
b

Intermediate Level Advanced Level
Intermediate Level Advanced Level


[('big data applications: machine learning at scale', 'Advanced Level', 3.8),
 ('introduction to machine learning', 'Intermediate Level', 4.7),
 ('practical reinforcement learning', 'Advanced Level', 4.1),
 ('statistical mechanics: algorithms and computations',
  'Mixed Level (Beginner Level)',
  4.8),
 ('deep learning in computer vision', 'Advanced Level', 3.9),
 ('matrix factorization and advanced techniques',
  'Mixed Level (Intermediate Level)',
  4.2),
 ('probabilistic graphical models 3: learning', 'Advanced Level', 4.6),
 ('big data essentials: hdfs, mapreduce and spark rdd',
  'Intermediate Level',
  4.0),
 ('convolutional neural networks', 'Intermediate Level', 4.9),
 ('probabilistic graphical models 1: representation', 'Advanced Level', 4.7),
 ('sequence models', 'Intermediate Level', 4.8),
 ('practical machine learning on h2o', 'Intermediate Level', 4.5),
 ('bayesian methods for machine learning',
  'Mixed Level (Intermediate Level)',
  4.6),
 ('visual perception for self-dr